# HAMT A100 Training - Google Colab

**Holographic Associative Memory Transformers**  
Train competitive, efficient LLMs with O(1) memory complexity

---

## 📋 Overview

This notebook trains HAMT models on Google Colab A100 GPU:
- **50M params**: Quick validation (~30 min)
- **100M params**: Competitive model (~6 hours)
- **200M params**: Large-scale model (~12 hours)

**Expected Results**: Match GPT-2 quality with 2-5× speedup and 20-50% fewer parameters

## 🚀 Step 1: Verify GPU

In [ ]:
# Check GPU availability
!nvidia-smi

import torch
print(f"\n🚀 GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else "")
print(f"   CUDA: {torch.version.cuda}" if torch.cuda.is_available() else "")

## 📦 Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install -q torch transformers datasets wandb accelerate

## 📥 Step 3: Clone Repository

In [ ]:
# Clone HAMT repository
!git clone https://github.com/codenlighten/holographic-ai.git
%cd holographic-ai

# Verify files
!ls -la

## 🔧 Step 4: Setup Environment

In [ ]:
# Add to Python path
import sys
sys.path.append('/content/holographic-ai')

# Test import
from src.hamt.model import HAMTModel
from src.hamt.config import HAMTConfig
print("✅ HAMT modules loaded successfully!")

## 🎯 Step 5A: Quick Validation (50M Model - 30 min)

**Purpose**: Validate setup before long training run  
**Time**: ~30 minutes  
**Dataset**: WikiText-2 (small)

In [ ]:
# Quick test configuration
!python train_gpu.py \
    --hidden_dim 384 \
    --hcm_dim 1536 \
    --num_layers 6 \
    --num_slots 16 \
    --dataset wikitext-2 \
    --max_seq_length 512 \
    --batch_size 32 \
    --num_epochs 3 \
    --learning_rate 3e-4 \
    --mixed_precision bf16 \
    --compile_model \
    --output_dir outputs/test_50m \
    --run_name hamt-test-50m

## 🏆 Step 5B: Main Training (100M Model - 6 hours)

**Purpose**: Competitive model for benchmarking  
**Time**: ~6 hours  
**Dataset**: WikiText-103 (standard benchmark)  
**Expected**: Perplexity < 28, competitive with GPT-2

In [ ]:
# Optional: Setup Weights & Biases for monitoring
import wandb
wandb.login()  # Enter your API key when prompted

In [ ]:
# Main training configuration (100M parameters)
!python train_gpu.py \
    --hidden_dim 512 \
    --hcm_dim 2048 \
    --num_layers 8 \
    --num_slots 16 \
    --dataset wikitext-103 \
    --max_seq_length 1024 \
    --batch_size 128 \
    --num_epochs 10 \
    --learning_rate 3e-4 \
    --mixed_precision bf16 \
    --compile_model \
    --use_wandb \
    --wandb_project hamt-competitive-llm \
    --output_dir outputs/hamt_100m \
    --run_name hamt-100m-wikitext103

## 🚀 Step 5C: Ambitious Training (200M Model - 12 hours)

**Purpose**: Large-scale competitive model  
**Time**: ~12 hours  
**Dataset**: WikiText-103  
**Expected**: Perplexity < 25, approaching GPT-2 Medium

In [ ]:
# Large model configuration (200M parameters)
!python train_gpu.py \
    --hidden_dim 768 \
    --hcm_dim 3072 \
    --num_layers 12 \
    --num_slots 16 \
    --dataset wikitext-103 \
    --max_seq_length 1024 \
    --batch_size 64 \
    --num_epochs 10 \
    --learning_rate 2e-4 \
    --mixed_precision bf16 \
    --compile_model \
    --gradient_checkpointing \
    --use_wandb \
    --wandb_project hamt-competitive-llm \
    --output_dir outputs/hamt_200m \
    --run_name hamt-200m-wikitext103

## 📊 Step 6: Monitor Training

In [ ]:
# Check training progress
!tail -20 outputs/hamt_100m/training.log

In [ ]:
# Monitor GPU utilization
!nvidia-smi

## 🔬 Step 7: Baseline Comparison

In [ ]:
# Compare HAMT with baseline transformers
!python compare_baselines.py \
    --hamt_checkpoint outputs/hamt_100m/checkpoints/best.pt \
    --baselines distilgpt2 gpt2 \
    --dataset wikitext-2 \
    --device cuda \
    --output_dir comparison_results

## 📈 Step 8: View Results

In [ ]:
# Load and display comparison results
import json
import pandas as pd

with open('comparison_results/comparison_results.json', 'r') as f:
    results = json.load(f)

# Create comparison table
df = pd.DataFrame(results).T
print("\n📊 Model Comparison Results:")
print("=" * 80)
print(df[['parameters', 'perplexity', 'throughput', 'peak_memory_gb']].to_string())
print("=" * 80)

## 💾 Step 9: Download Results

In [ ]:
# Package results for download
!zip -r results.zip \
    outputs/hamt_100m/checkpoints/best.pt \
    outputs/hamt_100m/checkpoints/best_config.json \
    comparison_results/ \
    outputs/hamt_100m/*.log

print("✅ Results packaged in results.zip")

In [ ]:
# Download results
from google.colab import files
files.download('results.zip')

## 🧪 Step 10: Test Generation (Optional)

In [ ]:
# Test text generation with trained model
import torch
from src.hamt.model import HAMTModel
from src.hamt.config import HAMTConfig
from transformers import GPT2Tokenizer

# Load model
checkpoint = torch.load('outputs/hamt_100m/checkpoints/best.pt')
config = HAMTConfig(**checkpoint['config'])
model = HAMTModel(config).cuda()
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Generate text
prompt = "Once upon a time"
input_ids = tokenizer.encode(prompt, return_tensors='pt').cuda()

with torch.no_grad():
    output = model.generate(input_ids, max_length=100, temperature=0.8)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print("\n📝 Generated Text:")
print("=" * 80)
print(generated_text)
print("=" * 80)

## 🐛 Troubleshooting

### Out of Memory (OOM)
```python
# Reduce batch size
--batch_size 64  # or 32

# Reduce sequence length
--max_seq_length 512

# Enable gradient checkpointing
--gradient_checkpointing
```

### Slow Training
```python
# Ensure BF16 is enabled (faster on A100)
--mixed_precision bf16

# Enable model compilation
--compile_model

# Increase batch size (if memory allows)
--batch_size 256
```

### Colab Disconnection
- Training auto-saves checkpoints every epoch
- Can resume from latest checkpoint (feature to be added)
- Or restart from scratch (checkpoints preserved in outputs/)

## 📊 Expected Results

### 100M Model (6 hours)
- **Perplexity**: ~25-28 on WikiText-103
- **Throughput**: 4000-6000 tokens/sec
- **Memory**: 6-8 GB VRAM
- **Quality**: Comparable to GPT-2 Small (124M)
- **Speedup**: 2-3× faster than GPT-2

### 200M Model (12 hours)
- **Perplexity**: ~22-25 on WikiText-103
- **Throughput**: 3000-5000 tokens/sec
- **Memory**: 12-16 GB VRAM
- **Quality**: Approaching GPT-2 Medium (355M)
- **Speedup**: 2-4× faster than GPT-2 Medium

## ✅ Success Criteria

**Minimum Success**:
- ✅ Training completes without OOM
- ✅ Loss decreases steadily
- ✅ Perplexity < 35
- ✅ Faster than baseline

**Good Success**:
- ✅ Perplexity < 30
- ✅ 2× faster than baseline
- ✅ Lower memory usage
- ✅ All checkpoints saved

**Excellent Success**:
- ✅ Perplexity < 28
- ✅ 3-5× faster than baseline
- ✅ Competitive with GPT-2
- ✅ Publication-ready results

## 📚 Additional Resources

- **GitHub**: https://github.com/codenlighten/holographic-ai
- **Whitepaper**: See `WHITEPAPER.md` in repo
- **Documentation**: See `README.md` and `USAGE_GUIDE.md`
- **Issues**: Report at GitHub Issues

---

**Good luck with your training! 🚀**